In [1]:
#Imports
from comet_ml import Experiment
experiment = Experiment(api_key="YkPEmantOag1R1VOJmXz11hmt", parse_args=False, project_name='SegNet')

COMET INFO: old comet version (1.0.44) detected. current: 1.0.51 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET WARNING: Failing to collect the installed os packages
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/yikeqicn/segnet/240d7408e904449ebf99e9e5cebc3d18



In [2]:
from os.path import join, basename, dirname
import tensorflow as tf
import shutil
import os
import sys
import argparse

import cv2
import numpy as np
import os
import time

from torch.utils.data import DataLoader, ConcatDataset, random_split#, SequentialSampler #yike: add SequentialSampler
import torch
import torchvision
import torchvision.transforms as transforms
from datasets import ArtPrint, ArtPrintNoIntsect
from Model_simpU_inhouse import *

home = os.environ['HOME']

In [3]:
#General Settings
parser = argparse.ArgumentParser()

# system basics
parser.add_argument("-name", default='segnet_no_intersect_1conv_64_unet_inhouse_30epoch', type=str, help="name of the log") #segnet_no_intersect_1conv_64_unet_inhouse_30epoch #debug model_intersect
parser.add_argument("-gpu", default='0', type=str, help="gpu numbers")

parser.add_argument("-train", default=True, help="train the NN", action="store_true")

parser.add_argument("-transfer",default=False, help="test the NN", action="store_true")

parser.add_argument("-test",default=False, help="test the NN", action="store_true")

# image and logistic parameters 
parser.add_argument("-image_h", default=32, type=int, help='image height') #('image_h', "360", """ image height """) 32
parser.add_argument("-image_w", default=128, type=int, help='image width')#('image_w', "480", """ image width """)128
#parser.add_argument("-image_h", default=360, type=int, help='image height') 
#parser.add_argument("-image_w", default=480, type=int, help='image width')

parser.add_argument("-image_c", default=1, type=int, help='image channel')#('image_c', "3", """ image channel (RGB) """)
parser.add_argument("-num_class", default=2, type=int, help='total class number')

# training hyperparam
parser.add_argument("-batch_size", default=10, type=int, help='batch_size')
parser.add_argument("-lrInit", default=1e-3, type=int, help='initial lr')
parser.add_argument("-lrDrop1", default=10, type=int, help='step to drop lr by 10 first time') # not sure
parser.add_argument("-lrDrop2", default=1000, type=int, help='step to drop lr by 10 sexond time') # not sure
parser.add_argument('-max_epoch',default=30, type=int,help='max epoch numbers')



# file paths
parser.add_argument('-ckpt_root', default="/home/classify_copy/misc_test/ckpt", type=str,help= "dir to store ckpt") # log_dir !!!!!
parser.add_argument('-data_root', default="/home/classify_copy/misc_test/datasets", type=str, help=" root to any data folder ")
parser.add_argument('-urlTranferFrom', default="", type=str, help=" archived model url ")


#args = parser.parse_args()
args = parser.parse_known_args()[0]

name = args.name

experiment.set_name(name)
experiment.log_parameters(vars(args))

os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

ckptroot = args.ckpt_root
args.ckptpath = join(ckptroot, name)
if args.name=='debug': shutil.rmtree(args.ckptpath, ignore_errors=True)
os.makedirs(args.ckptpath, exist_ok=True)


In [4]:
def checkArgs():
    if args.test:
        print('The model is set to Testing')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid testing dir: %s"%FLAGS.test_dir)
    elif args.transfer:
        print('The model is set to transfer learn from ckpt')
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Image dir: %s"%FLAGS.image_dir)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)
    else:
        print('The model is set to Training')
        print("Max training Iteration: %d"%args.max_epoch)
        print("Initial lr: %f"%args.lrInit)
        print("First Drop Steps: %i"%args.lrDrop1)
        print("Second Drop Steps: %i"%args.lrDrop2)
        print("Data root: %s"%args.data_root)
        print("Check point file: %s"%args.ckptpath)
        #print("CamVid Val dir: %s"%FLAGS.val_dir)

    print("Batch Size: %d"%args.batch_size)

In [5]:
#main function
def main():    
    checkArgs()
    if args.train:
        transform_train = transforms.Compose([
          transforms.Lambda(lambda img: cv2.resize(img, (args.image_w,args.image_h), interpolation=cv2.INTER_CUBIC)),
          transforms.Lambda(lambda img: np.expand_dims(img,3) ),
          #transforms.Lambda(lambda img: add_artifacts(img,args)),
          #transforms.Lambda(lambda img: cv2.transpose(img))
        ])
        #arprint=ArtPrint(args.data_root, transform=transform_train)
        arprint=ArtPrintNoIntsect(args.data_root, transform=transform_train)
        concat=arprint
        idxTrain = int( len(arprint)*0.9)
        trainset, testset = random_split(concat, [idxTrain, len(concat)-idxTrain])
        trainloader = DataLoader(trainset, batch_size=args.batch_size, shuffle=True, drop_last=True,num_workers=4)
        testloader = DataLoader(testset, batch_size=args.batch_size, shuffle=False, drop_last=False,num_workers=2)
        
        #weight gen
        pos_perc=sum(map(lambda x: np.sum(cv2.imread(x[1],cv2.IMREAD_GRAYSCALE)),trainset.dataset.samples))/(args.image_h*args.image_w*len(trainset.dataset.samples))
        neg_perc=1-pos_perc
        weight=np.array([pos_perc,neg_perc]) # just reverse
        print(weight)
        model=Model(args, experiment, loss_weight=weight, mustRestore=False)
        model.training(loader=trainloader, validateloader=testloader)
        
    else:
        pass # for now, leave it pass 
    

In [6]:
main()

The model is set to Training
Max training Iteration: 30
Initial lr: 0.001000
First Drop Steps: 10
Second Drop Steps: 1000
Data root: /home/classify_copy/misc_test/datasets
Check point file: /home/classify_copy/misc_test/ckpt/segnet_no_intersect_1conv_64_unet_inhouse_30epoch
Batch Size: 10
/home/classify_copy/misc_test/datasets/artifact_images_no_intersect already exists, skipping download
[0.05999042 0.94000958]
Instructions for updating:
Colocations handled automatically by placer.
GGG
(?, 32, 128, 1)
(?, 32, 128, 1)
(?, 32, 128, 64)
111111
(?, 16, 64, 64)
(?, 16, 64, 64)
22222
(?, 8, 32, 64)
(?, 8, 32, 64)
33333
(?, 4, 16, 64)
(?, 4, 16, 64)
44444
(?, 2, 8, 64)
(?, 2, 8, 64)
d4444444
(?, 4, 16, 64)
d333333
(?, 8, 32, 64)
d22222
(?, 16, 64, 64)
d111111
(?, 32, 128, 64)
cv
(?, 32, 128, 2)
(2,)
(?, 32, 128, 2)
loss: ()
INFO:tensorflow:Summary name conv_classifier/weight_loss (raw) is illegal; using conv_classifier/weight_loss__raw_ instead.
INFO:tensorflow:Summary name loss/cross_entrop

COMET ERROR: Failed to extract parameters from Estimator.init()


toto_loss_shape: Tensor("loss/total_loss:0", shape=(), dtype=float32)
Instructions for updating:
Use tf.cast instead.
Python: 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]
Tensorflow: 1.12.0-rc0
Ran global_variables_initializer
Epoch: 1  Training...


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys


TRAIN: Batch: 0.0 Loss: 0.13965543
accuracy = 0.906156
mean IU  = 0.547657
    class # 0 capture rate = 0.932243 
    class # 1 capture rate = 0.428571 
TRAIN: Batch: 0.003908998514580564 Loss: 0.058539383
accuracy = 0.097007
mean IU  = 0.050882
    class # 0 capture rate = 0.041444 
    class # 1 capture rate = 1.000000 
TRAIN: Batch: 0.007817997029161129 Loss: 0.047776088
accuracy = 0.059701
mean IU  = 0.029946
    class # 0 capture rate = 0.001685 
    class # 1 capture rate = 1.000000 
TRAIN: Batch: 0.011726995543741693 Loss: 0.045881264
accuracy = 0.056865
mean IU  = 0.028432
    class # 0 capture rate = 0.000000 
    class # 1 capture rate = 1.000000 
TRAIN: Batch: 0.015635994058322257 Loss: 0.049659096
accuracy = 0.078724
mean IU  = 0.039362
    class # 0 capture rate = 0.000000 
    class # 1 capture rate = 1.000000 
TRAIN: Batch: 0.019544992572902823 Loss: 0.0441904
accuracy = 0.048811
mean IU  = 0.024405
    class # 0 capture rate = 0.000000 
    class # 1 capture rate = 1.00

...

TRAIN: Batch: 0.574622781643343 Loss: 0.008587615
accuracy = 0.968358
mean IU  = 0.803037
    class # 0 capture rate = 0.967953 
    class # 1 capture rate = 0.974989 
TRAIN: Batch: 0.5785317801579235 Loss: 0.0074700215
accuracy = 0.971507
mean IU  = 0.828035
    class # 0 capture rate = 0.970471 
    class # 1 capture rate = 0.986868 
TRAIN: Batch: 0.5824407786725041 Loss: 0.0075496566
accuracy = 0.972070
mean IU  = 0.820117
    class # 0 capture rate = 0.971347 
    class # 1 capture rate = 0.983891 
TRAIN: Batch: 0.5863497771870847 Loss: 0.012425214
accuracy = 0.960643
mean IU  = 0.798651
    class # 0 capture rate = 0.960258 
    class # 1 capture rate = 0.965575 
TRAIN: Batch: 0.5902587757016652 Loss: 0.006712681
accuracy = 0.974972
mean IU  = 0.823949
    class # 0 capture rate = 0.974432 
    class # 1 capture rate = 0.984687 
TRAIN: Batch: 0.5941677742162458 Loss: 0.007729627
accuracy = 0.969645
mean IU  = 0.839196
    class # 0 capture rate = 0.968061 
    class # 1 capture ra

TRAIN: Batch: 0.7661637088577906 Loss: 0.008974886
accuracy = 0.973999
mean IU  = 0.838914
    class # 0 capture rate = 0.974199 
    class # 1 capture rate = 0.971081 
TRAIN: Batch: 0.7700727073723712 Loss: 0.0079119755
accuracy = 0.972163
mean IU  = 0.839697
    class # 0 capture rate = 0.971000 
    class # 1 capture rate = 0.987922 
TRAIN: Batch: 0.7739817058869518 Loss: 0.0074974946
accuracy = 0.965714
mean IU  = 0.806240
    class # 0 capture rate = 0.963996 
    class # 1 capture rate = 0.990843 
TRAIN: Batch: 0.7778907044015323 Loss: 0.0047095465
accuracy = 0.982322
mean IU  = 0.888705
    class # 0 capture rate = 0.981575 
    class # 1 capture rate = 0.992292 
TRAIN: Batch: 0.7817997029161129 Loss: 0.0064111506
accuracy = 0.973628
mean IU  = 0.795547
    class # 0 capture rate = 0.973020 
    class # 1 capture rate = 0.987042 
TRAIN: Batch: 0.7857087014306935 Loss: 0.005627675
accuracy = 0.978637
mean IU  = 0.859147
    class # 0 capture rate = 0.978059 
    class # 1 capture

TRAIN: Batch: 0.9577046360722383 Loss: 0.009346241
accuracy = 0.961487
mean IU  = 0.763160
    class # 0 capture rate = 0.961099 
    class # 1 capture rate = 0.968559 
TRAIN: Batch: 0.9616136345868188 Loss: 0.0063810498
accuracy = 0.975485
mean IU  = 0.835395
    class # 0 capture rate = 0.974879 
    class # 1 capture rate = 0.985476 
TRAIN: Batch: 0.9655226331013994 Loss: 0.0073713423
accuracy = 0.968135
mean IU  = 0.822321
    class # 0 capture rate = 0.966467 
    class # 1 capture rate = 0.991017 
TRAIN: Batch: 0.96943163161598 Loss: 0.0047807693
accuracy = 0.981542
mean IU  = 0.854125
    class # 0 capture rate = 0.981012 
    class # 1 capture rate = 0.991654 
TRAIN: Batch: 0.9733406301305606 Loss: 0.012413917
accuracy = 0.954119
mean IU  = 0.789456
    class # 0 capture rate = 0.951843 
    class # 1 capture rate = 0.980580 
TRAIN: Batch: 0.9772496286451411 Loss: 0.006090974
accuracy = 0.978464
mean IU  = 0.833735
    class # 0 capture rate = 0.978282 
    class # 1 capture ra

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  import sys


VALID: Total accuracy: 96.621733%. Class 0 capture: 96.508091%. Class 1 capture: 98.424372%
Character error rate not improved
Done with training at epoch 30 sigoptObservation=0.9736308116227038
